# Unsupervised Clustering on the MIMIC III database - queried for acute kidney injury dianosis

### Sina Dabiri
### sdabiri@emory.edu
#### Version 1.0

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# For this clustering project, the scikit learn PCA is going to be used from the decomposition library.
from sklearn import decomposition

In [4]:
"""
The kidney csv has been put in the project folder. The source of the MIMIC III is: 

https://physionet.org/content/mimiciii/1.4/
"""
kidney_df = pd.read_csv("KIDNEY.csv")
kidney_df

,subject_id,age_group,icd9_code
0,3,adult,5849
1,9,adult,5849
2,21,adult,5849
3,38,adult,5849
4,52,adult,5849
...,...,...,...
7678,99973,adult,5849
7679,99982,adult,5849
7680,99983,adult,5849
7681,99991,adult,5849


In [6]:
# Look for general stats and missing info
kidney_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7683 entries, 0 to 7682
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   subject_id  7683 non-null   int64 
 1   age_group   7683 non-null   object
 2   icd9_code   7683 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 180.2+ KB


In [9]:
# Dividing the dataset into it's X features and their true label Y
X = kidney_df[['subject_id', 'age_group']]
X

,subject_id,age_group
0,3,adult
1,9,adult
2,21,adult
3,38,adult
4,52,adult
...,...,...
7678,99973,adult
7679,99982,adult
7680,99983,adult
7681,99991,adult


In [ ]:
Y = 

In [1]:
pwd

'C:\\Users\\sinad\\OneDrive - Georgia Institute of Technology\\BMI 500 Bio Informatics\\wk7\\Dabiri_supervised_clustering'

In [5]:
mimiciii_path = "C:\Users\sinad\OneDrive - Georgia Institute of Technology\BMI 500 Bio Informatics\wk7\physionet.org\files\mimic3wdb\1.0"

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (Temp/ipykernel_31452/1436031810.py, line 1)

In [5]:
\d+ MIMICIII.ADMISSIONS;

SyntaxError: unexpected character after line continuation character (<ipython-input-5-f9b36644786b>, line 1)